### Requirements
- matplotlib
- numpy
- pandas
- seaborn
- joypy
- scipy
- requests
- (optional) tqdm

In [ ]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [ ]:
import pathlib
import sys
PATH = pathlib.Path().absolute().resolve()

%matplotlib inline
import math
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as tk
import matplotlib.colors as clr
import joypy as jp
import seaborn as sns
    
sns.set(style="whitegrid", font_scale=1.4)
pd.set_option('display.float_format', '{:.2f}'.format)
COLORS = sns.color_palette()

In [ ]:
import ipywidgets
from scipy.signal import savgol_filter

In [ ]:
import json
import requests
from dateutil.parser import parse as date_parse
from datetime import datetime, timedelta
from time import sleep
try:
    from tqdm import tqdm
except ImportError:
    # tqdm placeholder
    class tqdm:
        def __init__(self, it=None): 
            self.it=it
        def __enter__(self): 
            return self
        def __exit__(self, *args, **kwargs): 
            pass
        def __iter__(self): 
            return iter(self.it)
        def update(*args): 
            pass

In [ ]:
from IPython.display import Javascript, display
from ipywidgets import widgets

def run_all(ev):
    display(Javascript('IPython.notebook.execute_cells_below()'))

In [ ]:
user_id = int(input("Type user id here (number): "))

In [ ]:
matches_url = lambda offset, limit=100: f"https://api.2018.halite.io/v1/api/user/{user_id}/match?offset={offset}&limit={limit}"

In [ ]:
print("Loading data... It may take a while...")
data = []
chunk = True
step = 100
i = 0
with tqdm() as progress:
    while chunk:
        url = matches_url(step * i, step)
        resp = requests.get(url)
        try:
            chunk = resp.json()
        except json.JSONDecodeError:
            print(resp.text)
            sleep(1)
        else:            
            i += 1
            data.extend(chunk)
            progress.update(step)

In [ ]:
data[0]

In [ ]:
print("Creating dataset...")

In [ ]:
filtered_data = []
for item in data:
    if item['challenge_id'] is not None:
        continue
    p = item["players"][str(user_id)]
    stats = item["stats"]["player_statistics"][p["player_index"]]
    row = {        
        "game_id": item["game_id"],
        "time_played": date_parse(item["time_played"]),
        "mu": p["mu"],
        "sigma": p["sigma"],
        "leaderboard_rank": p["leaderboard_rank"],
        "version": p["version_number"],
        "num_players": float(len(item["players"])),
        "total_h": stats["final_production"] / 1000,
        "mined_h": stats["total_mined"] / 1000,
        "map_size": item["map_height"]
    } 
    filtered_data.append(row)

In [ ]:
df = pd.DataFrame(filtered_data)
df['timestamp'] = df.time_played.apply(lambda dt: int(dt.timestamp()))
df.set_index("time_played", inplace=True)
df.sort_index(inplace=True, ascending=False)

In [ ]:
tmp = df[df.num_players == 3].index
if len(tmp):
    df.drop(tmp, inplace=True)

In [ ]:
df['num_games'] = -1
for version in sorted(df.version.unique()):
    df_tmp = df[df.version == version]
    for i, ix in enumerate(df_tmp.index[::-1]):
        ix = ix.isoformat()
        df.loc[ix, 'num_games'] = i + 1

In [ ]:
df.head()

In [ ]:
_start_date = ipywidgets.DatePicker(description='Pick a start date', disabled=False, style={'description_width': 'initial'})
_start_date.observe(run_all, names='value')

In [ ]:
display(_start_date)

In [ ]:
start_timestamp = int(datetime.combine(_start_date.value, datetime.min.time()).timestamp()) if _start_date.value else None
df_filtered = df[df.timestamp > start_timestamp] if start_timestamp else df
df_filtered = df_filtered[df_filtered.sigma < 2]
df_filtered.describe()[['mu', 'sigma', 'num_players', 'total_h', 'mined_h']].T

In [ ]:
versions = sorted(df_filtered.version.unique())

In [ ]:
fig = plt.figure(figsize=(20, 10), dpi=200)
ax = fig.add_subplot(111)

# Split mu sequnce by version, filtered it and join 
Mean = None
for version in versions:
    df_tmp = df_filtered[df_filtered.version == version]
    window = min(df_tmp.mu.count() // 8 * 2 + 1, 101)
    tmp = savgol_filter(df_tmp.mu, window, 5) if window > 10 else df_tmp.mu
    Mean = tmp if Mean is None else np.concatenate((tmp, Mean))
    ax.axvline(df_tmp.index.min(), linestyle='--', linewidth=1, color='black')

Sigma = df_filtered.sigma
Top = Mean + 2 * Sigma
Bottom = Mean - 2 * Sigma

ax.fill_between(df_filtered.index, Bottom, Top, alpha=.1)
for Y in (Top, Bottom):
    ax.plot(df_filtered.index, Y, c=COLORS[0], linewidth=.5)
ax.plot(df_filtered.index, Mean, c=COLORS[0])
ax.plot(df_filtered.index, df_filtered.mu, '.', c=COLORS[0])

ax.set_xlim(df_filtered.index.min(), df_filtered.index.max())
ax.set_ylabel("Mu +- 2 * Sigma (Score)")
ax.grid(which='minor', axis='x', dashes=(5, 5), linewidth=.5)
ax.grid(which='minor', axis='y', dashes=(5, 5), linewidth=.5)
ax.minorticks_on()
fig.tight_layout()
plt.show()

In [ ]:
ax = sns.lmplot(x='num_games', y='num_players', data=df_filtered, height=10, scatter_kws=dict(s=.5), y_jitter=.1)
ax = ax.axes[0, 0]
ax.grid(which='minor', axis='x', dashes=(5, 5), linewidth=.5)
ax.grid(which='minor', axis='y', dashes=(5, 5), linewidth=.5)
ax.minorticks_on()

In [ ]:
fig = plt.figure(figsize=(20, 10), dpi=200)
ax = fig.add_subplot(111)
for version in versions:    
    df_tmp = df_filtered[df_filtered.version == version]
    sns.regplot(x=df_tmp.timestamp, y=df_tmp.mined_h, scatter_kws=dict(s=.1), ax=ax, truncate=True, color=COLORS[0])
    sns.regplot(x=df_tmp.timestamp, y=df_tmp.total_h, scatter_kws=dict(s=.1), ax=ax, truncate=True, color=COLORS[1])
    ax.axvline(df_tmp.timestamp.min(), linestyle='--', linewidth=1, color='black')
ax.grid(which='minor', axis='x', dashes=(5, 5), linewidth=.5)
ax.grid(which='minor', axis='y', dashes=(5, 5), linewidth=.5)
ax.minorticks_on()
m, s = df_filtered.mined_h.mean(), df_filtered.mined_h.std()
ax.set_ylim(m - 2 * s, m + 2 * s)

In [ ]:
axes = sns.pairplot(df_filtered, vars=['total_h', 'mined_h'], hue="num_players", height=7).axes
for row in axes:
    for ax in row:
        ax.grid(which='minor', axis='x', dashes=(5, 5), linewidth=.5)
        ax.grid(which='minor', axis='y', dashes=(5, 5), linewidth=.5)
        ax.minorticks_on()

In [ ]:
plt.figure(figsize=(16,10), dpi= 80)
fig, axes = jp.joyplot(
    df_filtered, column='mined_h', by=["num_players", "map_size"],
    figsize=(14,10), linewidth=1, fill=False, grid='y'
)
for ax in axes:
    ax.grid(which='major', axis='x')
    ax.grid(which='minor', axis='x', dashes=(5, 5), linewidth=.5)
    ax.minorticks_on()